In [4]:
from abc import ABC, abstractmethod
    

class Pair:

    _from_ = None
    _to = None

    def __init__(self, from_, to):
        self._from_ = from_
        self._to = to
    
    def __eq__(self, object):
        return self._from_ == object._from_ and self._to == object._to
    
    def __hash__(self):
        return 0
    

class Expression(ABC):  # an interface
    

    @abstractmethod
    def reduce(self, bank, to : str): # -> Money
        pass


    @abstractmethod
    def __add__(self, addend): # -> Expression
        pass


class Money(Expression):    # implement the interface

    _amount = None     # In Java, protected
    _currency = None   # In Java, protected


    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency
    
    
    def __eq__(self, object):
        # In Java, casting to Money first, then:
        return self._amount == object._amount and self.currency() == object.currency()


    @staticmethod
    def dollar(amount): # -> Money
        return Money(amount, "USD")
    

    @staticmethod
    def franc(amount):  # -> Money
        return Money(amount, "CHF")
    

    def __mul__(self, multiplier) -> Expression:
        return Money(self._amount * multiplier, self._currency)


    def currency(self):
        return self._currency
    

    def __str__(self):
        return str(self._amount) + " " + self._currency
    

    def __add__(self, addend: Expression) -> Expression:
        return Sum(self, addend)
    

    def reduce(self, bank, to : str): # -> Money
        rate = bank.rate(self._currency, to)
        return Money(self._amount / rate, to)
    

class Sum(Expression):      # implement interface

    augend: Expression | None = None
    addend: Expression | None = None


    def __init__(self, augend: Expression, addend: Expression):
        self.augend = augend
        self.addend = addend

    
    def reduce(self, bank, to: str):
        amount : int = self.augend.reduce(bank, to)._amount \
            + self.addend.reduce(bank, to)._amount
        return Money(amount, to)
    

class Bank:


    _rates : dict[Pair, int] = {}


    def reduce(self, source: Expression, to: str) -> Money:
        return source.reduce(self, to)
    

    def rate(self, from_: str, to: str) -> int:
        if from_ == to:
            return 1
        return self._rates[Pair(from_, to)]
    

    def addRate(self, from_: str, to: str, rate: int):
        self._rates[Pair(from_, to)] = rate

1. ~~\\$5 + 10 CHF = \\$10 if rate is 2:1~~
2. ~~\\$5 * 2 = \\$10~~.
3. ~~make `amount` private~~.
4. ~~Dollar side-effects?~~
5. Money rounding?
6. ~~`equals()`~~.
7. Equal null.
8. Equal object.
9. ~~5 CHF * 2 = 10 CHF~~.
10. ~~Dollar/Franc duplication~~.
11. ~~Common equals~~.
12. ~~Common times~~.
13. ~~Compare Francs with Dollars~~.
14. ~~Currency?~~
15. Delete testFrancMultiplication?
16. ~~\\$5 + \\$5 = \\$10~~
17. Return `Money` from \\$5 + \\$5
18. ~~`Bank.reduce(Money)`~~
19. ~~Reduce `Money` with conversation~~
20. ~~`Reduce(Money, String)`~~
21. **`Sum.plus`**
22. `Expression.times`

We need to implement `Sum.plus()` to finish `Expression.plus`, and then we need
`Expression.times()`, and then we’re finished with the whole example.

In [2]:
def testSumPlusMoney():
    five_bucks: Expression = Money.dollar(5)
    ten_francs: Expression = Money.franc(10)
    bank: Bank = Bank()
    bank.addRate("CHF", "USD", 2)
    sum_: Expression = Sum(five_bucks, ten_francs) + five_bucks
    result: Money = bank.reduce(sum_, "USD")
    assert result == Money.dollar(15)


testSumPlusMoney()

TypeError: Can't instantiate abstract class Sum with abstract method __add__

In [5]:
class Sum(Expression):      # implement interface

    augend: Expression | None = None
    addend: Expression | None = None


    def __init__(self, augend: Expression, addend: Expression):
        self.augend = augend
        self.addend = addend

    
    def reduce(self, bank, to: str):
        amount : int = self.augend.reduce(bank, to)._amount \
            + self.addend.reduce(bank, to)._amount
        return Money(amount, to)
    

    def __add__(self, addend: Expression) -> Expression:
        return Sum(self, addend)
    

testSumPlusMoney()

1. ~~\\$5 + 10 CHF = \\$10 if rate is 2:1~~
2. ~~\\$5 * 2 = \\$10~~.
3. ~~make `amount` private~~.
4. ~~Dollar side-effects?~~
5. Money rounding?
6. ~~`equals()`~~.
7. Equal null.
8. Equal object.
9. ~~5 CHF * 2 = 10 CHF~~.
10. ~~Dollar/Franc duplication~~.
11. ~~Common equals~~.
12. ~~Common times~~.
13. ~~Compare Francs with Dollars~~.
14. ~~Currency?~~
15. Delete testFrancMultiplication?
16. ~~\\$5 + \\$5 = \\$10~~
17. Return `Money` from \\$5 + \\$5
18. ~~`Bank.reduce(Money)`~~
19. ~~Reduce `Money` with conversation~~
20. ~~`Reduce(Money, String)`~~
21. ~~`Sum.plus`~~
22. **`Expression.times`**

In [6]:
def testSumTimes():
    five_bucks: Expression = Money.dollar(5)
    ten_francs: Expression = Money.franc(10)
    bank: Bank = Bank()
    bank.addRate("CHF", "USD", 2)
    sum_: Expression = Sum(five_bucks, ten_francs) * 2
    result: Money = bank.reduce(sum_, "USD")
    assert result == Money.dollar(20)


testSumTimes()

TypeError: unsupported operand type(s) for *: 'Sum' and 'int'

In [7]:
class Expression(ABC):  # an interface
    

    @abstractmethod
    def reduce(self, bank, to : str): # -> Money
        pass


    @abstractmethod
    def __add__(self, addend): # -> Expression
        pass


    @abstractmethod
    def __mul__(self, multiplier): # -> Expression
        pass
    

class Sum(Expression):      # implement interface

    augend: Expression | None = None
    addend: Expression | None = None


    def __init__(self, augend: Expression, addend: Expression):
        self.augend = augend
        self.addend = addend

    
    def reduce(self, bank, to: str):
        amount : int = self.augend.reduce(bank, to)._amount \
            + self.addend.reduce(bank, to)._amount
        return Money(amount, to)
    

    def __add__(self, addend: Expression) -> Expression:
        return Sum(self, addend)
    

    def __mul__(self, multiplier) -> Expression:
        return Sum(self.augend * multiplier, self.addend * multiplier)
    

testSumTimes()

1. ~~\\$5 + 10 CHF = \\$10 if rate is 2:1~~
2. ~~\\$5 * 2 = \\$10~~.
3. ~~make `amount` private~~.
4. ~~Dollar side-effects?~~
5. Money rounding?
6. ~~`equals()`~~.
7. Equal null.
8. Equal object.
9. ~~5 CHF * 2 = 10 CHF~~.
10. ~~Dollar/Franc duplication~~.
11. ~~Common equals~~.
12. ~~Common times~~.
13. ~~Compare Francs with Dollars~~.
14. ~~Currency?~~
15. Delete testFrancMultiplication?
16. ~~\\$5 + \\$5 = \\$10~~
17. Return `Money` from \\$5 + \\$5
18. ~~`Bank.reduce(Money)`~~
19. ~~Reduce `Money` with conversation~~
20. ~~`Reduce(Money, String)`~~
21. ~~`Sum.plus`~~
22. ~~`Expression.times`~~